In [2]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from tqdm import tqdm
import matplotlib.pyplot as plt
from PIL import Image

# ✅ EarlyStopping 類別
class EarlyStopping:
    def __init__(self, patience=8, verbose=True, delta=0):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_acc = None
        self.early_stop = False
        self.delta = delta
        self.best_model_state = None

    def __call__(self, acc, model):
        if self.best_acc is None:
            self.best_acc = acc
            self.best_model_state = model.state_dict()
        elif acc < self.best_acc + self.delta:
            self.counter += 1
            if self.verbose:
                print(f"⚠️ 早停計數器: {self.counter}/{self.patience}")
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            if self.verbose:
                print(f"✅ 驗證準確度提升: {self.best_acc:.2f} → {acc:.2f}，重置早停計數器")
            self.best_acc = acc
            self.best_model_state = model.state_dict()
            self.counter = 0

# 設定設備
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 設定圖片路徑與參數
dataset_path = "corp_augmented_data"
batch_size = 1024
IMG_SIZE = 64

# 圖片預處理
transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# 資料集與資料分割
dataset = datasets.ImageFolder(root=dataset_path, transform=transform)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print(f"類別對應: {dataset.class_to_idx}")

# 模型定義
class DeepCoffeeBeanCNN(nn.Module):
    def __init__(self):
        super(DeepCoffeeBeanCNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),  # 64x64 → 64x64
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(32, 32, kernel_size=3, padding=1),  # 64x64
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),  # 64x64 → 32x32

            nn.Conv2d(32, 64, kernel_size=3, padding=1),  # 32x32
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),  # 32x32
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),  # 32x32 → 16x16

            nn.Conv2d(64, 128, kernel_size=3, padding=1),  # 16x16
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),  # 16x16
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),  # 16x16 → 8x8
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),                # 128 x 8 x 8 = 8192
            nn.Linear(128 * 8 * 8, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, 2)  # 分兩類
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x


# 初始化模型
model = CoffeeBeanCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
early_stopper = EarlyStopping(patience=8, verbose=True)

# 訓練迴圈
num_epochs = 80
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", leave=True)

    for images, labels in progress_bar:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        correct += predicted.eq(labels).sum().item()
        total += labels.size(0)
        progress_bar.set_postfix(loss=loss.item(), acc=100 * correct / total)

    train_acc = 100 * correct / total
    print(f"Epoch {epoch+1}/{num_epochs} - Loss: {running_loss:.4f}, Accuracy: {train_acc:.2f}%")

    # 驗證階段
    model.eval()
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            val_correct += (predicted == labels).sum().item()
            val_total += labels.size(0)
    val_acc = 100 * val_correct / val_total
    print(f"📉 驗證準確度: {val_acc:.2f}%")

    # 檢查是否早停
    early_stopper(val_acc, model)
    if early_stopper.early_stop:
        print("🛑 觸發早停，停止訓練")
        break

# 儲存最佳模型
torch.save(early_stopper.best_model_state, "coffee_bean_cnn.pth")
print("✅ 模型已儲存為最佳驗證表現!")

# 測試模型
model.load_state_dict(early_stopper.best_model_state)  # 使用最佳權重
model.eval()
correct = 0
total = 0
progress_bar = tqdm(test_loader, desc="Testing", leave=True)

with torch.no_grad():
    for images, labels in progress_bar:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)
        progress_bar.set_postfix(acc=100 * correct / total)

test_acc = 100 * correct / total
print(f"📊 測試準確度: {test_acc:.2f}%")


類別對應: {'bad': 0, 'good': 1}


Epoch 1/80: 100%|██████████| 4/4 [00:03<00:00,  1.15it/s, acc=56.9, loss=0.647]


Epoch 1/80 - Loss: 2.6624, Accuracy: 56.93%
📉 驗證準確度: 67.79%


Epoch 2/80: 100%|██████████| 4/4 [00:02<00:00,  1.67it/s, acc=65.4, loss=0.646]


Epoch 2/80 - Loss: 2.5962, Accuracy: 65.40%
📉 驗證準確度: 67.79%
✅ 驗證準確度提升: 67.79 → 67.79，重置早停計數器


...

Epoch 42/80 - Loss: 1.6178, Accuracy: 81.34%
📉 驗證準確度: 73.51%
⚠️ 早停計數器: 7/8


Epoch 43/80: 100%|██████████| 4/4 [00:02<00:00,  1.68it/s, acc=80.7, loss=0.404]


Epoch 43/80 - Loss: 1.6231, Accuracy: 80.72%
📉 驗證準確度: 72.95%
⚠️ 早停計數器: 8/8
🛑 觸發早停，停止訓練
✅ 模型已儲存為最佳驗證表現!


Testing: 100%|██████████| 1/1 [00:00<00:00,  2.06it/s, acc=73]

📊 測試準確度: 72.95%


In [2]:
print(f"訓練集圖片數量：{len(train_dataset)} 張")
print(f"測試集圖片數量：{len(test_dataset)} 張")
print(f"訓練集 Batch 數量：{len(train_loader)} 個 batch（每 batch {batch_size} 張）")
print(f"測試集 Batch 數量：{len(test_loader)} 個 batch（每 batch {batch_size} 張）")


訓練集圖片數量：3564 張
測試集圖片數量：891 張
訓練集 Batch 數量：4 個 batch（每 batch 1024 張）
測試集 Batch 數量：1 個 batch（每 batch 1024 張）
